In [9]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
from sklearn.preprocessing import LabelEncoder
import tensorflow as tf
import keras

In [2]:
df = pd.read_csv('nlp/dataset.csv')
df = df.drop(df[df['Language'] != 'en'].index)
df = df.drop(df[df['Label'] == 'litigious'].index)
df = df.drop(df[df['Label'] == 'uncertainty'].index)
df = df.drop(['Language'], axis=1)
df = df[:10000]
df

,Text,Label
3,Rwanda is set to host the headquarters of Unit...,positive
5,It sucks for me since I'm focused on the natur...,negative
8,Social Security. Constant political crises dis...,negative
9,@FilmThePoliceLA A broken rib can puncture a l...,negative
10,@jacobringenwald @aKeithWatts @CountDankulaTV ...,negative
...,...,...
18906,@grimsbeard @wewerenishiki I mean it's cute......,positive
18908,@nappynapkin Good night 🌙 sleep well 😴 and don...,positive
18909,@_SilverCrystal_ Dang poor Jett looks like the...,negative
18910,Strive is a good game tho.\nAnd i wish it beca...,positive


In [15]:
from sklearn.model_selection import train_test_split
x = df['Text']
y = df['Label']
le = LabelEncoder()
y = le.fit_transform(y)
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.20)

In [16]:
train = tf.data.Dataset.from_tensor_slices((x_train, y_train))
test = tf.data.Dataset.from_tensor_slices((x_test, y_test))

In [17]:
train = train.batch(64).prefetch(tf.data.AUTOTUNE)
test = test.batch(64).prefetch(tf.data.AUTOTUNE)

In [18]:
vectorizer = keras.layers.TextVectorization(max_tokens=1000)
vectorizer.adapt(train.map(lambda text, label: text))
model = keras.Sequential([
    vectorizer,
    keras.layers.Embedding(input_dim=len(vectorizer.get_vocabulary()), output_dim=64, mask_zero=True),
    keras.layers.Bidirectional(keras.layers.LSTM(64)),
    keras.layers.Dense(64, activation='relu'),
    keras.layers.Dense(1)
    
    
])
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 text_vectorization_1 (TextV  (None, None)             0         
 ectorization)                                                   
                                                                 
 embedding_1 (Embedding)     (None, None, 64)          64000     
                                                                 
 bidirectional_1 (Bidirectio  (None, 128)              66048     
 nal)                                                            
                                                                 
 dense_2 (Dense)             (None, 64)                8256      
                                                                 
 dense_3 (Dense)             (None, 1)                 65        
                                                                 
Total params: 138,369
Trainable params: 138,369
Non-tr

In [19]:
model.compile(loss=keras.losses.BinaryCrossentropy(from_logits=True),optimizer=keras.optimizers.Adam(1e-4),metrics=['accuracy'])

In [20]:
history = model.fit(train, epochs=32, validation_data=test, validation_steps=30)

Epoch 1/32
125/125 [==============================] - 14s 67ms/step - loss: 0.6895 - accuracy: 0.4879 - val_loss: 0.6833 - val_accuracy: 0.5083
Epoch 2/32
125/125 [==============================] - 6s 51ms/step - loss: 0.5930 - accuracy: 0.5859 - val_loss: 0.4243 - val_accuracy: 0.8115
Epoch 3/32
125/125 [==============================] - 7s 52ms/step - loss: 0.2955 - accuracy: 0.9170 - val_loss: 0.2123 - val_accuracy: 0.9536
Epoch 4/32
125/125 [==============================] - 7s 53ms/step - loss: 0.1519 - accuracy: 0.9721 - val_loss: 0.1401 - val_accuracy: 0.9682
Epoch 5/32
125/125 [==============================] - 8s 62ms/step - loss: 0.1054 - accuracy: 0.9776 - val_loss: 0.1515 - val_accuracy: 0.9615
Epoch 6/32
125/125 [==============================] - 8s 62ms/step - loss: 0.0986 - accuracy: 0.9764 - val_loss: 0.1456 - val_accuracy: 0.9620
Epoch 7/32
125/125 [==============================] - 8s 62ms/step - loss: 0.0648 - accuracy: 0.9844 - val_loss: 0.0999 - val_accuracy: 0.970

In [21]:
model.evaluate(test)

32/32 [==============================] - 1s 26ms/step - loss: 0.1869 - accuracy: 0.9680


[0.18688012659549713, 0.9679999947547913]